# Lista 3 - Lucas Carneiro

In [3]:
import os

import nltk
from nltk.corpus import machado
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import PortugueseStemmer

import string

from collections import defaultdict
from collections import OrderedDict

import numpy as np
from numpy.linalg import norm

import matplotlib.pylab as plt

from gensim import corpora, models, similarities

import pandas as pd

import seaborn as sns

### Exercício 1

O Primeiro passo é organizar o corpus a ser utilizado.

A base escolhida foram os documentos de Machado.

In [4]:
textos = [machado.raw(id) for id in machado.fileids()]
swu = stopwords.words('portuguese') + list (string.punctuation)
stemmer = PortugueseStemmer()

textos_limpos = []
for texto in textos:
    tlimpo = [stemmer.stem(token.lower()) for token in WordPunctTokenizer().tokenize(texto) if token not in swu]
    textos_limpos.append(tlimpo)

Com os dados limpos criaremos um corpus adequado a utlização pelo gensim

In [5]:
dictionary = corpora.Dictionary(textos_limpos)
corpus = [dictionary.doc2bow(text) for text in textos_limpos]

A partir do corpus gerado, construiremos o modelo LSI

In [6]:
#Construção de modelo TFIDF
mod_tfidf = models.TfidfModel(corpus)
corpus_tfidf = mod_tfidf[corpus]

#Construção de modelo LSI
mod_lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=200)
corpus_lsi = mod_lsi[corpus_tfidf] 

O assunto escolhido será o sexto tópico de nosso modelo.

O critério utilizado foi escolher o tópico com interpretação mais intuitiva

In [28]:
topico = 5
mod_lsi.show_topic(topico,10)

[('cecíl', -0.56734219735659397),
 ('luís', 0.42830216709079988),
 ('alves', 0.20137512502482588),
 ('venânci', -0.13207811179097584),
 ('carlot', -0.12329779767923273),
 ('major', 0.11894631801783583),
 ('magalhã', -0.11637546023256877),
 ('albert', 0.11481660552969455),
 ('tibúrci', -0.10886793623477728),
 ('marcelin', 0.10287118355012496)]

Constriremos o nosso conjunto relevante como sendo os 25 documentos mais influenciados pelo tópico escolhido

In [70]:
lista = []
for i,texto in enumerate(corpus_lsi):
    lista.append((texto[topico][1],i))

R_relevantes = []
n_relevantes = 25
for i in sorted(lista,reverse = True)[:n_relevantes]:
    R_relevantes.append(i[1])

In [71]:
R_relevantes = set(R_relevantes)
print(R_relevantes)

{1, 133, 15, 158, 163, 38, 45, 55, 57, 59, 190, 191, 64, 192, 193, 71, 205, 81, 210, 83, 82, 84, 93, 224, 235}


Construiremos também a nossa consulta utilizando os 10 termos mais pertinentes ao nosso tópico

In [39]:
t_relevantes = 10
q_consulta = []
for t in mod_lsi.show_topic(topico,10):
    q_consulta.append(t[0])

In [41]:
print(q_consulta)

['cecíl', 'luís', 'alves', 'venânci', 'carlot', 'major', 'magalhã', 'albert', 'tibúrci', 'marcelin']


### Exercício 2

Começaremos por gerar o índice invertido do nosso corpus

In [42]:
indice = defaultdict(lambda:set([]))
for tid,t in enumerate(textos_limpos):
    for term in t:
        indice[term].add(tid)

Para esse exercício será utilizada a seguinte função de busca booleana que retorna todos os documentos que possuem pelo menos um dos termos pesquisados

In [46]:
def busca_adv(search, indice):

    resultado = indice[search[0]]
    for token in search[1:]:
            resultado = resultado | indice[token]
    
    return resultado

In [66]:
resp_boo = busca_adv(q_consulta,indice)
resp_boo = set(resp_boo)
print(resp_boo)

{0, 1, 2, 4, 5, 11, 12, 13, 15, 16, 18, 21, 23, 24, 25, 26, 27, 28, 29, 33, 34, 36, 38, 39, 40, 43, 45, 48, 49, 50, 51, 52, 54, 55, 57, 59, 60, 61, 63, 64, 66, 67, 71, 76, 80, 81, 82, 83, 84, 92, 93, 95, 96, 105, 107, 110, 113, 120, 121, 122, 128, 139, 140, 151, 154, 158, 161, 162, 163, 165, 168, 171, 174, 175, 176, 179, 182, 183, 185, 187, 188, 190, 191, 192, 193, 205, 210, 217, 218, 219, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 235, 236, 237, 238, 240, 241, 242, 244, 245}


Quanto ao tf_idf, faremos a busca da seguinte forma:

In [50]:
T = nltk.TextCollection(textos_limpos)

tfidf_matrix = np.zeros((len(textos_limpos),len(q_consulta)))

for j, termo in enumerate(q_consulta):
    for i, texto in enumerate(textos_limpos):
        tfidf_matrix[i,j] = T.tf_idf(termo,texto)
        
Mtfidf_Norm = np.array([r/norm(r) if norm(r) !=0 else np.zeros(len(r)) for r in tfidf_matrix])

Para definir os documentos encontrados utilizaremos o método de truncagem com valor estabelecido em (>=) 0,4

In [82]:
def ordem(q,MN):
    return [np.dot(q,r) for r in MN]

vetor_tfidf = np.array([T.tf_idf(w,q_consulta) for w in q_consulta])
vetor_tfidf /= norm(vetor_tfidf)
resp_tfidf = ordem(vetor_tfidf,Mtfidf_Norm)

vtfidf = filter(lambda x : x[0]!=0.0, zip(resp_tfidf,range(len(textos_limpos))))

temp_tfidf = sorted(vtfidf, reverse=True)

trunc = 0.3
resp_tfidf = []
for i in temp_tfidf:
    if i[0]>= trunc:
        resp_tfidf.append(i[1])

In [83]:
resp_tfidf = set(resp_tfidf)
print(resp_tfidf)

{0, 2, 11, 140, 13, 139, 15, 26, 158, 161, 33, 34, 165, 38, 168, 48, 52, 182, 183, 185, 57, 187, 191, 193, 66, 67, 82, 217, 219, 93, 95, 224, 225, 223, 235, 236, 237, 241, 120, 122}


Agora reutilizaremos a função para cálculo de Revocação e Precisão

In [73]:
def RevPre(parcial,geral):
    
    #parcial = 'documentos encontrados'
    #geral = 'documentos relevantes'
    
    rev = len(parcial & geral)/len(geral)
    pre = len(parcial & geral)/len(parcial)
    
    print('Revocação = ' + str(rev))
    print('Precisão = ' + str(pre))
    
    #return rev,pre

In [84]:
print('Resultados para a busca booleana')
RevPre(resp_boo,R_relevantes)
print('\n')
print('Resultados para a busca por tf_idf')
RevPre(resp_tfidf,R_relevantes)

Resultados para a busca booleana
Revocação = 0.96
Precisão = 0.21818181818181817


Resultados para a busca por tf_idf
Revocação = 0.4
Precisão = 0.25


### Exercício 3

Construindo a função de busca probabilística

In [131]:
def prob_busca(query, indice, relevantes, N_docs = 246):
    
    #query = consulta a ser realizada
    #indice = indice invertido
    #relevantes = conjunto de documentos atualmente considerados relevantes
    #N_docs = número de documentos presente no corpus
    
    query_tok = query #[stemmer.stem(token.lower())  for token in WordPunctTokenizer().tokenize(query) if token not in swu]
    
    ct = []
    S = len(relevantes)
    N = N_docs
    
    for termo in query_tok:
        s = len(indice[termo] & relevantes)
        df = len(indice[termo])
        
        up = s/(S - s +0.5)
        down =(df - s)/(N - df - S + s -0.5)
        
        ct.append(np.log(up/down))
    
    RSVs = []
    for doc in range(N):
        rsv = 0
        for pos,termo in enumerate(query_tok):
            if doc in indice[termo]:
                rsv += ct[pos]
        RSVs.append((rsv,doc))
        
    resultado = sorted(RSVs,reverse=True)
    
    return resultado

In [132]:
temp_prob = prob_busca(q_consulta,indice,R_relevantes)
print(temp_prob[:10])

[(9.5992284489099688, 191), (9.5297251576714537, 193), (8.0967590183874858, 57), (7.9042516335644954, 82), (7.8347483423259785, 1), (7.0256565673047344, 13), (6.899111083505626, 237), (6.899111083505626, 15), (6.568748768711731, 229), (6.090019308484381, 93)]


C:\Users\Casa\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in log


Note-se que o parâmetro 'relevantes' é na verdade o conjunto de documentos considerados relevantes naquele momento, sendo portanto um conjunto que deve ser atualizado conforme a resposta do usuário.

O parâmetro 'N_docs' é a quantidade de documentos no corpus utilizado sendo então praticamente fixo.
Pode ser obtido através do 'indice' mas considerei que retardaria a função desnecessáriamente.

Para calcular a Revocação e a Precisão, novamente usaremos o método de truncagem, considerando como resposta os documentos com RSV acima de 1.

In [112]:
trunc = 1
resp_prob = []
for i in temp_prob:
    if i[0] >= trunc:
        resp_prob.append(i[1])
        
resp_prob = set(resp_prob)
print(resp_prob)

{0, 1, 2, 128, 4, 12, 13, 15, 21, 23, 151, 25, 24, 27, 28, 29, 158, 161, 34, 163, 36, 165, 38, 39, 168, 40, 45, 175, 176, 49, 51, 182, 183, 55, 57, 59, 187, 188, 61, 191, 192, 193, 66, 64, 63, 190, 67, 71, 205, 81, 82, 210, 83, 84, 217, 219, 92, 93, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 96, 105, 235, 237, 238, 240, 113, 244, 120, 122}


In [113]:
RevPre(resp_prob,R_relevantes)

Revocação = 0.96
Precisão = 0.3037974683544304


Calcular a probabilidade p_termo para cada termo de nossa consulta

In [121]:
p_termo = []
S = len(R_relevantes)
for termo in q_consulta:
    s = len(indice[termo] & R_relevantes)
    p_termo.append((termo,s/S))

In [120]:
print(p_termo)
print('\n')
print(mod_lsi.show_topic(topico,10))

[('cecíl', 0.04), ('luís', 0.84), ('alves', 0.24), ('venânci', 0.0), ('carlot', 0.16), ('major', 0.36), ('magalhã', 0.08), ('albert', 0.2), ('tibúrci', 0.04), ('marcelin', 0.16)]


[('cecíl', -0.56734219735659397), ('luís', 0.42830216709079988), ('alves', 0.20137512502482588), ('venânci', -0.13207811179097584), ('carlot', -0.12329779767923273), ('major', 0.11894631801783583), ('magalhã', -0.11637546023256877), ('albert', 0.11481660552969455), ('tibúrci', -0.10886793623477728), ('marcelin', 0.10287118355012496)]


Ainda que os vetores sejam aparentemente distintos é possivel perceber uma correlação em termos de ordem de significância.

Os termos com menor relevância para o tópico, também apresentam uma probabilidade menor

### Exercício 4

Repetindo o procedimento o exercício anterior.

Definimos a nova função de busca

In [138]:
def okapi_busca(query, indice, corpo):
    
    #query = consulta a ser realizada
    #indice = indice invertido
    #corpo = corpus
    
    query_tok = query #[stemmer.stem(token.lower())  for token in WordPunctTokenizer().tokenize(query) if token not in swu]
    
    N = len(corpo)
    
    L = []
    for texto in corpo:
        L.append(len(texto))
        
    L_ave = sum(L)/N
    k1 = 1.2
    b = 0.75
    
    RSVs = []
    for doc in range(N):
        rsv = 0
        
        for termo in query_tok:
            if doc in indice[termo]:
                tf = 1 #pode ser alterado para melhor representar a frequencia
            df = len(indice[termo])
        rsv += np.log(N/df)*((k1 + 1)*tf)/(k1*(1 - b + (b*(L[doc]/L_ave))) + tf)
        
        RSVs.append((rsv,doc))
        
    resultado = sorted(RSVs,reverse=True)
    
    return resultado

In [134]:
temp_okapi = okapi_busca(q_consulta,indice, textos_limpos)
print(temp_okapi[:10])

[(6.2393542810351423, 199), (6.2027710326330876, 211), (6.2007879168888991, 204), (6.1522693329462017, 197), (6.1490184201635358, 170), (6.1354020203837338, 177), (6.1051446611116287, 169), (6.069480689933819, 201), (6.0618925523896925, 209), (6.0336052313313404, 212)]


Aplicando método de truncagem para definir a resposta da função de busca

In [136]:
trunc_okapi = 1
resp_okapi = []
for i in temp_okapi:
    if i[0] >= trunc_okapi:
        resp_okapi.append(i[1])
        
resp_okapi = set(resp_okapi)
print(resp_prob)

{0, 1, 2, 128, 4, 12, 13, 15, 21, 23, 151, 25, 24, 27, 28, 29, 158, 161, 34, 163, 36, 165, 38, 39, 168, 40, 45, 175, 176, 49, 51, 182, 183, 55, 57, 59, 187, 188, 61, 191, 192, 193, 66, 64, 63, 190, 67, 71, 205, 81, 82, 210, 83, 84, 217, 219, 92, 93, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 96, 105, 235, 237, 238, 240, 113, 244, 120, 122}


E, finalmente, calculando a Revocação e a Precisão

In [137]:
RevPre(resp_okapi,R_relevantes)

Revocação = 0.96
Precisão = 0.09836065573770492


Nota-se ganhos consideráveis no algoritmo Okapi em relação aos outros